# Day & Ross Freight: Sales Opportunity Win Prediction, an Analysis of Salesforce Data

\begin{itemize}
\item One
\item Two
\end{itemize}


${\displaystyle y_{i}=\beta _{0}+\beta _{1}x_{i1}+\cdots +\beta _{p}x_{ip}+\varepsilon _{i}=\mathbf {x} _{i}^{\mathsf {T}}{\boldsymbol {\beta }}+\varepsilon _{i},\qquad i=1,\ldots ,n,}{\displaystyle y_{i}=\beta _{0}+\beta _{1}x_{i1}+\cdots +\beta _{p}x_{ip}+\varepsilon _{i}=\mathbf {x} _{i}^{\mathsf {T}}{\boldsymbol {\beta }}+\varepsilon _{i},\qquad i=1,\ldots ,n,}$

In [3]:
import numpy as np
import pandas as pd

## Introduction

Headquartered in Hartland, New Brunswick, Day & Ross Freight Inc. is a Canadian transportation and
logistics company that was founded more than 65 years ago. It has grown from a few trucks hauling goods
in eastern Canada into one of the nation’s largest national transportation providers. It is a wholly owned
subsidiary of McCain Foods Limited (Day & Ross Transportation Group, 2017).

Companies that apply data driven decision making perform better than those in which decision makers
rely on subjective models based on experience (Provost and Fawcett, 2013). Day & Ross Inc. have used
Salesforce.com’s Sales Cloud for several years, and through it, sales representatives and executives at the
company can access various reports and dashboards summarizing data on the sales pipeline (Finelli, 2018). A
move up the Analytics Maturity Model (TDWI, 2016) through the application of machine learning techniques
could allow Day & Ross Inc. to better leverage their Salesforce.com data.

This report presents the results of the the machine learning models that were applied to predict whether an
opportunity would be won or lost and the factors that have influence. Two models were built using logistic
regression and random forest. In addition the data was explored, visualized, cleaned and appropriate features
were selected.

## Business Problem

A sales pipeline is a method of selling based on the fundamental principles of the sales process. 
It describes the sequence of steps that a sales person takes from first contact with a potential 
customer, to qualifying that prospect as a lead, to the validation of that lead into an opportunity, 
and moving through further stages until it is closed. Day & Ross monitors this sales 
pipeline using Salesforce.com, the world's largest CRM software vendor (Columbus, 2016). 
Figure 1 shows a Kanban Chart displayed in a typical Salesforce.com's Sales Cloud application 
showing the current pipeline of opportunities and their values.

The use of machine learning technology in business is predicted to double by the end of 2018 
(Deloitte, 2017). The major CRM players are all betting that machine learning and artificial 
intelligence will be the next major industry disruptor, and have already implemented machine 
learning software in their CRM systems, including
Oracle's Adaptive Intelligence, Microsoft's Cortana Intelligence Suite, and Salesforce's 
Enstein AI (Evans, 2018).

Day & Ross, much like any company, would like to increase the number of opportunities that they win. 
There are many demands on a sales representative's time and by scoring opportunities with the 
probability of success a rep can concentrate on those deals that are more likely to succeed 
and waste less time on those with very little chance of success. In addition, by knowing 
which factors are most correlated with winning an opportunity a business can focus on 
those activities that are most likely to move the needle.

\begin{figure}
\centering
\includegraphics[width=1]{pic/kanban.png}
\caption{Kanban chart of open opportunities in Salesforce.com’s Sales Cloud.}
\label{image-myimage}
\end{figure}


## Data

Salesforce.com data is contained in standard and custom objects, which can be thought of in database terms
as tables. All records for the objects relevant to this project were exported and provided to the author using
Salesforce.com’s Apex Data Loader (for events and tasks), or by means of a custom report.

Table 1 Shows the Salesforce.com objects used in this study. Opportunity is the main object of interest
and contains fields such as the stage, the reason, the expected revenue and the start and close date of the
opportunity. The other tables are related to the opportunity table through lookup fields (this is akin to the
referential integrity of a database).

The user table, related through the User Id, shows details about the owner of the opportunity, and the
account table, related through the Account Id, shows details about the associated account. The event and
task tables are used interchangeably and display details about the activities a sales representative has with a
client, for example, if they call or email them. These activity tables are related to the opportunity through
the WhatId field. The contact table shows information about the people who work at the business that’s
related to the opportunity.

| Tables   |      Are      |  Cool |
|----------|:-------------:|------:|
| col 1 is |  left-aligned | \$1,600 |
| col 2 is |    centered   |   \$12 |
| col 3 is | right-aligned |   \$1 |

|Salesforce Object| No. of Records|Description|
|:-: |-|:---|
|Opportunity |16,913 | All opportunities, their names, potential revenue, stage, start date and closed date, type, and reason.|
|Account |53,985 |The account associated with the opportunity - each opportunity references only one account.|
|Contact |53,161 | The contact from the client’s company referenced by the opportunity.|
|User    |180| The Day & Ross employee who owns the opportunity - each opportunity is owned by only one user.|
|Event   |64,275| The activities related to an opportunity, or the associated account, for example a call or email.|
|Task     |770,532| Tasks are used interchangeably with events so the tables were combined.|

|One|Two|
|:-|---|
|2 | 3|

opportunities = pd.read_csv("data/AM - Opportunities.csv", \
                            encoding="latin-1")

In [26]:
accounts = pd.read_csv("data/AM - Accounts.csv",\
                      encoding="latin-1")

In [29]:
contacts = pd.read_csv("data/AM - Contacts.csv",\
               encoding="latin-1")

In [31]:
contacts.head(3)

,Account ID,Account Name,Contact ID,First Name,Last Name,No Longer Available,Mailing Street,Mailing City,Mailing State/Province,Mailing Country,Mailing Zip/Postal Code,Email,Description,Created Date
0,0010B00001v8sB7,Hermatz Corpy LLC,0030B0000278KdA,Marina,Jin,0,553 Alta Road Unit 3,San Diego,California,USA,92154,hermatzcorpyllc@hotmail.com,NaN,1/11/2017
1,001U0000007ferU,Columbia Remtec Mfg,003U0000008MlPU,Frank,Putland,0,NaN,NaN,NaN,NaN,NaN,fputland@columbia-remtec.com,NaN,15/12/2011
2,001U000000WS6Qt,ABS CANADA,003U000000XEUOt,Judith,Riley,0,1401 Meyerside Drive Unit # 2,Mississauga,ON,CAN,NaN,judith.riley@sulzer.com,NaN,18/12/2012


In [15]:
opportunities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16913 entries, 0 to 16912
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Opportunity ID                  16913 non-null  object 
 1   Opportunity Name                16913 non-null  object 
 2   Type                            16913 non-null  object 
 3   Stage                           16913 non-null  object 
 4   Reason                          15867 non-null  object 
 5   Revenue Potential               16873 non-null  float64
 6   Created Date                    16913 non-null  object 
 7   Close Date                      16913 non-null  object 
 8   Service Offering Name           15670 non-null  object 
 9   Annual Revenue Potential        16873 non-null  float64
 10  User ID                         16913 non-null  object 
 11  Opportunity Owner               16913 non-null  object 
 12  Owner Role                      

In [33]:
opportunities['Reason'].unique()

array([nan, 'No Equipment', 'Cross-functional between divisions',
       'Relationship', 'Price', 'Needs other Features',
       'Not Qualified Correctly', 'Unknown', 'Solution',
       'Cancelled Project', 'Prior competitor relationship',
       'Change in D/M', 'Contract w/ Current Solution',
       'Prior Negative D&R Experience', 'Speed of Resolution',
       'Incorrect Fit', 'Online Shipping', 'Canadian', 'Terms',
       'Credit Issue', 'Risk too high', 'Driver', 'VIP'], dtype=object)

In [22]:
deals = pd.read_csv("data/deals.csv")

In [23]:
deals

,Unnamed: 0,Stage,Type,PotRevenue,AccCalls,AgeInDays,TotOpCalls,UserTotCalls,UserTenure,AccountAge,NumContacts,NumOppEmails,NumAccEmails,CanOrNot,Win
0,1,Closed Lost,New Business,4800,2,101,7,41063,2079,534,1,5,11,1,0
1,2,Closed Lost,New Business,24000,1,64,2,1907,950,534,1,0,0,1,0
2,3,Closed Won,New Business,2400,7,52,3,28836,1890,534,2,1,6,1,1
3,4,Closed Lost,Expanded,20000,17,45,0,12081,1771,534,8,0,9,1,0
4,5,Closed Won,New Business,20000,12,37,3,12081,1771,534,8,0,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16862,16909,Closed Lost,New Business,43200,5,111,6,5125,1771,836,1,0,0,0,0
16863,16910,Closed Lost,New Business,28800,2,67,2,5125,1771,836,1,0,0,0,0
16864,16911,Closed Won,New Business,15000,2,176,2,3794,1771,835,1,0,0,1,1
16865,16912,Closed Won,New Business,6000,19,45,0,41063,2079,835,3,0,53,1,1


In [24]:
opportunities

,Opportunity ID,Opportunity Name,Type,Stage,Reason,Revenue Potential,Created Date,Close Date,Service Offering Name,Annual Revenue Potential,User ID,Opportunity Owner,Owner Role,Opportunity Owner: Department,Account Number: Record ID,Account Number: Account Number,Account ID,Account Name
0,0060B00000eO1v0,MITYBILT - BC PROJECT (TL),Expanded,Proposal,NaN,50000.0,31/12/2017,31/03/2018,SCS TL / LTL,50000.04,0050B000007Fk56,Ainsley McWhirter,Freight West Account Executive Prairies,West Freight,a0CU000000DYoz9,82623,001U000000WSABn,MITYBILT PRODUCTS
1,0060B00000eO1YB,COLE INTERNATIONAL - US TL - HUMBLE TX TO HALI...,Expanded,Closed Lost,No Equipment,40000.0,31/12/2017,22/01/2018,NaN,40000.00,005U00000027Cb4,Miriam Watts,Maritimes AE: Freight,Maritimes Freight,a0CU000000QdXmI,67466,001U0000007fy5P,COLE INTERNATIONAL - HALIFAX
2,0060B00000eO2AD,LTL US ITR America,New Business,Closed Won,Cross-functional between divisions,10800.0,31/12/2017,31/12/2017,LTL USA,10800.00,005U0000001Dcbw,Alvin Rodricks,Freight Ontario: Account Executive,Ontario Freight,a0C0B00000re88m,127325,0010B00001w2sZ5,ITR America
3,0060B00000eO295,TL MILLER & SMITH,New Business,Closed Won,Relationship,24000.0,31/12/2017,31/12/2017,Asset TL BH/Needed Lanes,24000.00,005U0000001Dcbw,Alvin Rodricks,Freight Ontario: Account Executive,Ontario Freight,a0CU000000DYlnL,22349,001U000000WSAAG,MILLER & SMITH FOODS INC
4,0060B00000eO1YL,DALCAM SANITATION - DOMESTIC LTL - ON & QC INT...,Expanded,Negotiation,NaN,10632.0,31/12/2017,22/03/2018,LTL Domestic,10632.00,005U00000027Cb4,Miriam Watts,Maritimes AE: Freight,Maritimes Freight,a0C0B00000jTNQc,143949,001U000000WS7yL,DALCAM SANITATION SOLUTIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16908,006U000000BnOOs,GCR LTL Newfoundland and Quebec,New Business,Closed Won,Relationship,14400.0,1/8/2013,28/03/2014,LTL Domestic,14400.00,005U0000001Dcbx,Brent Ryan,Maritimes AE: Freight,Maritimes Freight,a0CU000000DZZ9I,66527,001U000000WSV6A,GCR Tire Centre - Moncton
16909,006U000000AVd7K,Clippe from Va to Ont and West- US LTL,Expanded,Closed Lost,Cancelled Project,96000.0,1/5/2013,9/8/2013,LTL USA,96000.00,005U0000001Dcbx,Brent Ryan,Maritimes AE: Freight,Maritimes Freight,a0CU000000E4F0s,127904,001U000000WSIi9,WHEELS CLIPPER
16910,006U000000AVfcp,Ex Whitby,Expanded,Closed Lost,Cancelled Project,100800.0,1/5/2013,24/09/2013,LTL Domestic,100800.00,005U0000001Dcar,Michelle Barrett,Freight Ontario: Account Executive,Ontario Freight,a0CU000000DYpqG,3720,001U000000WS6p1,ATLANTIC PACKAGING PRD
16911,006U000000AVd5e,Keltic i/b USA LTL to NFLD,Expanded,Closed Lost,Price,12000.0,1/5/2013,23/09/2013,LTL USA,12000.00,005U00000027CbE,Shawn Lizotte,Maritimes AE: Freight,Maritimes Freight,a0CU000000DYlc8,34326,001U000000WS9Wf,KELTIC TRANSPORTATION Head Office MTN


In [25]:
a = pd.read_csv("data/AM - Accounts.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfb in position 2: invalid start byte

$
{\displaystyle \mathbf {y} =X{\boldsymbol {\beta }}+{\boldsymbol {\varepsilon }},\,}{\displaystyle \mathbf {y} =X{\boldsymbol {\beta }}+{\boldsymbol {\varepsilon }},\,}$